In [24]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time


In [25]:
v, f = igl.read_triangle_mesh('data/hand.off')
labels = np.load('data/hand.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

In [26]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [27]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [28]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    return target_pos
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''

' (Optional) Register this function to perform interactive deformation\npos_f.deformer = position_deformer\n'

In [29]:
## Widget UI

p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

# 1: Removal of high-frequency details

In [30]:
# express Lw, M-1
# add constraints

import scipy.sparse as sp
v_smooth = handle_vertex_positions.copy()

Lw = igl.cotmatrix(v, f)
M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
M_inverse = sp.diags(1 / M.data)

A = Lw @ M_inverse @ Lw
Aff = A[labels==0, :][:, labels==0]
Afc = A[labels==0, :][:, labels>0]
xc = v_smooth[labels>0, :]
x = sp.linalg.spsolve(Aff, -Afc @ xc)
v_smooth[labels==0] = x
p = mp.plot(v_smooth, f, c=labels, shading = {"wireframe": True, "flat": False})
p = mp.plot(v_smooth, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

# 2: Deforming the smooth mesh

In [31]:
# 1. di = vis - viB (B is the smoothed one) so di is just the same shape with v
# 2. rotate the the details di with mesh B  
#     2.1 Calculate normal ni (for surface B)  
#     2.2 Project all neighboring vertices to the tangent plane (perpendicular to ni)  
#     2.3 Find neighbor j* for which projected edge (i, j) is longest. Normalize this edge vector and call it xi.  
#     2.4 Construct yi using the cross product, completing orthonormal frame (xi, yi, ni) 
# 3. di = dixxi + diyyi + dinni  inner products.


def construct_mesh_details(v_sm):
    d = v - v_sm
    # print(d.shape, v.shape)
    normals = igl.per_vertex_normals(v_sm, f)
    neighs = igl.adjacency_list(f)
    j_longest = np.zeros(v.shape[0], dtype=np.int16)
    di = np.zeros_like(v)

    for i in range(v.shape[0]):
        ni = normals[i]
        i_j = v_sm[neighs[i]] - v_sm[i]
        projected_edges = i_j - i_j @ ni[:, None] * ni
        neighs_index = np.argmax(np.linalg.norm(projected_edges, axis=1))
        j_longest[i] = neighs[i][neighs_index]
        xi = projected_edges[neighs_index]
        xi /= np.linalg.norm(xi)
        yi = np.cross(ni, xi)
        di[i] = np.array([d[i] @ xi, d[i] @ yi, d[i] @ ni])
    return di, j_longest


xc_2 = handle_vertex_positions[labels>0, :]
x_2 = sp.linalg.spsolve(Aff, -Afc @ xc_2)
v_smooth_2 = handle_vertex_positions
v_smooth_2[labels==0] = x_2
p_2 = mp.plot(v_smooth_2, f, c=labels, shading = {"wireframe": True, "flat": False})
p_2 = mp.plot(v_smooth_2, f, c=labels)

#caculate for mesh B
di_B, j_longest_B = construct_mesh_details(v_smooth)
#caculate for mesh B'
di_B_2, j_longest_B_2 = construct_mesh_details(v_smooth_2)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

# 3: Transferring high-frequency details to the deformed surface  


In [32]:
# 1. The unit vertex normal  
# 2. The normalized projection of one of 's outgoing edges onto the tangent plane defined by the vertex normal. A stable choice is the edge whose projection onto the tangent plane is longest.  
# 3. The cross-product between (1) and (2)

def transfer_details(v_sm, di, j_longest):
    di_new = np.zeros_like(di)
    v_detailed = v_sm.copy()
    ni = igl.per_vertex_normals(v_sm, f)
    edges = v_sm[j_longest] - v_sm
    xi = np.zeros_like(ni)
    xi = edges - ni*(np.sum(edges * ni, axis=1).reshape(-1, 1))
    xi /= np.linalg.norm(xi).reshape(-1, 1)
    yi = np.cross(ni, xi)
    for i in range(v.shape[0]):
        di_new[i] = di[i] @ np.vstack((xi[i], yi[i], ni[i]))
    v_detailed += di_new    
    return v_detailed

v_detailed_2 = transfer_details(v_smooth_2,di_B_2,j_longest_B_2)
p_3 = mp.plot(v_detailed_2, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>